# Segmenting and Clustering Neighborhoods in Toronto

## Question 1

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#### Import text from wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')


#### Convert text into readable format

In [3]:
# Get all the details
tabulka = soup.find("table", {"class" : "wikitable sortable"})
tabulka.prettify()
gdp_table_data = tabulka.tbody.find_all("tr") 

#### Store data in list

In [4]:
table_data = []
data = {}
for item in gdp_table_data:
    # Get all the headings of Lists
    headings = []
    for td in gdp_table_data[0].find_all("th"):
        # Extract the headings and remove whitespaces
        headings.append(td.text.replace('\n', ' ').strip())

    # Get all the rows of table
    t_row = {}
    row_data=[]
    for tr in item.find_all("td"):
        for data in tr:
            text = data.replace('\n', ' ').strip()
            row_data.append(text)
    # Append to table    
    table_data.append(row_data)

#### Convert text into DataFrame

In [5]:
# Convert to table and drop the first row
table= pd.DataFrame(table_data,columns=headings)
table = table.iloc[1:]

#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


In [6]:
table = table[table['Borough']!='Not assigned']

#### More than one neighborhood can exist in one postal code area. 
 For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: 
Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a 
comma as shown in row 11 in the above table.

In [7]:
table = table.groupby('Postal code').agg(lambda x : ','.join(set(x))).reset_index()
table.Neighborhood = table.Neighborhood.str.replace('/',',')


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [8]:
table.loc[table['Neighborhood']=='Not assigned','Neighborhood'] = table['Borough']

#### Shape of dataframe

In [9]:
table.shape

(103, 3)